[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/okada-tak/deep-learning-from-scratch-2/blob/master/notebooks/ch05.ipynb)

# 5章 リカレントニューラルネットワーク(RNN) のまとめ
- RNNはループする経路があり、それによって「隠れ状態」を内部に記憶することができる
- RNNのループ経路を展開することで、複数のRNNレイヤがつながったネットワークと解釈することができ、通常の誤差逆伝播法によって学習することができる（＝BPTT）
- 長い時系列データを学習する場合は、適当な長さでデータのまとまりを作り－これを「ブロック」という－、ブロック単位でBPTTによる学習を行う（＝Truncated BPTT）
- Truncated BPTTでは逆伝播のつながりのみを切断する
- Truncated BPTTでは順伝播のつながりは維持するため、データは”シーケンシャル”に与える必要がある
- 言語モデルは、単語の羅列を確率として解釈する
- RNNレイヤを利用した条件付き言語モデルは、（理論的には）それまで登場した単語の情報を記憶することができる

単純なフィードフォワード・ネットワークでは、時系列データの性質（パターン）を十分に学習することができない

## 5.1 確率と言語モデル
### 5.1.1 word2vecを確率の視点から眺める